In [ ]:
from datascience import *
import numpy as np
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

## Classification ##

In [ ]:
patients = Table.read_table('breast-cancer.csv').drop('ID')
patients.show(5)

In [ ]:
def distance(pt1, pt2):
    """Return the distance between two points, represented as arrays"""
    return np.sqrt(sum((pt1 - pt2)**2))

def row_distance(row1, row2):
    """Return the distance between two numerical rows of a table"""
    return distance(np.array(row1), np.array(row2))

def distances(training, example):
    """Compute distance between example and every row in training.
    Return training augmented with Distance column"""
    distances = make_array()
    attributes = training.drop('Class')
    for row in attributes.rows:
        distances = np.append(distances, row_distance(row, example))
    return training.with_column('Distance', distances)

def closest(training, example, k):
    """Return a table of the k closest neighbors to example"""
    return distances(training, example).sort('Distance').take(np.arange(k))

def majority_class(topk):
    """Return the class with the highest count"""
    return topk.group('Class').sort('count', descending=True).column(0).item(0)

def classify(training, example, k):
    "Return the majority class among the k nearest neighbors of example"
    return majority_class(closest(training, example, k))

In [ ]:
attributes = patients.drop('Class')
attributes.show(3)

In [ ]:
example123 = attributes.row(123)
example123

In [ ]:
classify(patients.exclude(123), example123, 5)

In [ ]:
patients.take(123)

## Evaluation ##

In [ ]:
patients.num_rows

In [ ]:
shuffled = patients.sample(with_replacement=False) 
training_set = shuffled.take(np.arange(342))
test_set  = shuffled.take(np.arange(342, 683))

In [ ]:
def evaluate_accuracy(training, test, k):
    """Return the proportion of correctly classified examples 
    in the test set"""
    test_attributes = test.drop('Class')
    num_correct = 0
    for i in np.arange(test.num_rows):
        c = classify(training, test_attributes.row(i), k)
        num_correct = num_correct + (c == test.column('Class').item(i))
    return num_correct / test.num_rows

In [ ]:
evaluate_accuracy(training_set, test_set, 5)

In [ ]:
evaluate_accuracy(training_set, test_set, 7)

In [ ]:
evaluate_accuracy(training_set, training_set, 1)

## Decisions ##

In [ ]:
# np.array(list) converts list to an array
# provided all the elements of list are of the same type

n = 100
second = round(n * 0.6)
third = round(n * 0.4)

year = np.array(['Second'] * second + ['Third'] * third)
major = np.array(['Declared'] * (round(second * 0.5)) + ['Undeclared'] * (round(second * 0.5)) + \
                 ['Declared'] * (round(third * 0.8))  + ['Undeclared'] * (round(third * 0.2)))
                 
students = Table().with_columns(
    'Year', year,
    'Major', major
)
students.show(3)

In [ ]:
students.pivot('Major', 'Year')

In [ ]:
# Chance of junior, given that they have declared
# P(junior | declared)

32 / 62

In [ ]:
# P(sophomore | declared)

30 / 62

In [ ]:
(0.6 * 0.5) / ((0.6 * 0.5) + (0.4 * 0.8))

In [ ]:
(0.001 * 1) / ((0.001 * 1) + (0.999 * 0.05))

In [ ]:
n = 10000
disease = round(n * 0.001)
no_disease = round(n * 0.999)

status = np.array(['Disease'] * disease + ['No disease'] * no_disease)
result = np.array(['Test +'] * (disease) + ['Test +'] * (round(no_disease * 0.05))  + \
                 ['Test -'] * (round(no_disease * 0.95)))
                 
persons = Table().with_columns(
    'Status', status,
    'Test Result', result
)
persons.show(3)

In [ ]:
persons.pivot('Test Result', 'Status')

In [ ]:
500 + 9490

## Using Subjective Prior Probabilities ##

In [ ]:
(0.1 * 1) / ((0.1 * 1) + (0.999 * 0.05))

In [ ]:
(0.5 * 1) / ((0.5 * 1) + (0.5 * 0.05))

In [ ]:
def create_population(prior_disease_prob, n):
    disease = round(n * prior_disease_prob)
    no_disease = round(n * (1 - prior_disease_prob))

    status = np.array(['Disease'] * disease  +  ['No disease'] * no_disease)
    result = np.array(['Test +'] * (disease) + ['Test +'] * (round(no_disease * 0.05))  + \
                 ['Test -'] * (round(no_disease * 0.95)))
                 
    t = Table().with_columns(
    'Status', status,
    'Test Result', result
    )
    return t.pivot('Test Result', 'Status')

In [ ]:
create_population(0.5, 10000)

In [ ]:
5000 / (5000 + 250)